# Connect to bio.tools API

## First, get West-Life tools

In [33]:
# 
import requests

# a GET request
response = requests.get('https://dev.bio.tools/api/tool/?collectionID=West-Life&format=json',
    # could  "Accept": "application/json;charset=UTF-8"
)
assert 200 == response.status_code
btservices = response.json()['list']


## Convert to einfracentral format

In [50]:
import json

# options, 
eiservices = []
for btservice in btservices:
    eiservice = {
        # required terms 
        'name': btservice['name'],
        'url': btservice['homepage'],
        'order': btservice['download'][0]['url'] if btservice['download'] else btservice['homepage'], 
        "providers": ['West-Life'],

        # Required, but no good match in bio.tools
        'symbol': 'https://about.west-life.eu/network/west-life/templates/westlife/images/west-life.png', 
        "category": "Category-Software", "subcategory": "Subcategory-Tool",
        "places": ["Place-WW"],
        
        # length limit in einfacentral
        'funding' : "EC funding ",
        
        # terms that map fairly well
        # could register all 'providers': btservice['collectionID'], # nearest approximation
        'version': btservice['version'][0],
        "lastUpdate": btservice['lastUpdate'],
        'description': btservice['description'],
        "languages": ['Language-'+btservice['language']] if btservice['language'] else ["Language-en"],
        'tags': [ t['term'] for t in btservice['topic']],
        
        # defaults, revised below if possible
        "serviceLevelAgreement": "https://auth.west-life.eu/aai/AUP.pdf", 
        'helpdesk': 'https://about.west-life.eu/network/west-life/documentation',
        'options': '',
        "termsOfUse":  [btservice['license']] or ["https://auth.west-life.eu/aai/AUP.pdf"], 

    }
    for f in btservice['function']:
        for op in f['operation']:
            eiservice['options'] = eiservice['options'] +op['term']+', '
    if btservice['function']:
        for f in btservice['function']:
            if f['note']:
                tagline = f['note']
                eiservice['tagline'] = tagline if len(tagline)<100 else tagline[0:97]+'...'
                break;
    if 'Mature'==btservice['maturity']:
        eiservice["lifeCycleStatus"] = "LifeCycleStatus-Production"
        eiservice["trl"]= "TRL-9"
    elif 'Emerging'==btservice['maturity']:
        eiservice["lifeCycleStatus"] = "LifeCycleStatus-Beta"
        eiservice["trl"]= "TRL-7" # permissible levels are 9,8,7
    else:
        print('maturity', btservice['maturity'])
        
    for d  in btservice['documentation']:
        if 'Citation instructions' == d['type']:
            pass
        elif 'Terms of Use' == d['type']:
            eiservice['termsOfUse'] = d['url']
            eiservice['serviceLevelAgreement'] = d['url']
    for d  in btservice['credit']:
        # unfortunately "max length for 'funding' is 500 chars"
        #if  'Funding agency' == d['typeEntity']:
            # eiservice['funding'] = eiservice['funding']+', '+d['name']
        if  'Primary Contact' in d['typeRole']:
            eiservice['helpdesk'] = d['url']

    # validate it        
    HEADERS = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    response = requests.post('http://beta.einfracentral.eu/api/service/validate', 
        data=json.dumps(eiservice), headers=HEADERS)

    assert 200 == response.status_code, response.json()
    eiservices.append(eiservice)
    print(eiservice['name'])
'validated'

DisVis web portal
PowerFit web portal
HADDOCK web portal
Virtual Folder for Structural Biology Projects
PDB_REDO server
Protein Crystallisation Construct Designer


'validated'

In [49]:
eiservices

[{'name': 'DisVis web portal',
  'url': 'http://milou.science.uu.nl/services/DISVIS',
  'order': 'https://github.com/haddocking/disvis',
  'providers': ['West-Life'],
  'symbol': 'https://about.west-life.eu/network/west-life/templates/westlife/images/west-life.png',
  'category': 'Category-Software',
  'subcategory': 'Subcategory-Tool',
  'places': ['Place-WW'],
  'funding': 'EC funding ',
  'version': '1.0',
  'lastUpdate': '2018-09-18T12:06:18Z',
  'description': 'DisVis allows you to visualizes and quantify the information content of distance restraints between macromolecular complexes.\nIt performs a full and systematic 6 dimensional search of the three translational and rotational degrees of freedom to determine the number of complexes consistent with the restraints. In addition, it outputs the percentage of restraints being violated and a density that represents the center-of-mass position of the scanning chain corresponding to the highest number of consistent restraints at every